In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("GOOGLE_API_KEY")


In [3]:
import fitz  # PyMuPDF
import os

# PDF path (can be anywhere)
file_path = r"C:\Users\Raghul mani M\Downloads\compliance_docs\compliance_docs\upload_nic_2018_5c94b4f64e25f.pdf"
pdf = fitz.open(file_path)

# Current working directory instead of __file__
project_root = os.getcwd()
output_root = os.path.join(project_root, "output_images")

# Extract PDF name (without extension) for subfolder
pdf_name = os.path.splitext(os.path.basename(file_path))[0]
output_dir = os.path.join(output_root, pdf_name)
os.makedirs(output_dir, exist_ok=True)

# Convert each page to image
for page_number in range(len(pdf)):
    page = pdf[page_number]
    pix = page.get_pixmap(dpi=300)
    output_path = os.path.join(output_dir, f"page_{page_number+1}.png")
    pix.save(output_path)
    print(f"Saved {output_path}")


Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_1.png
Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_2.png
Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_3.png
Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_4.png
Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_5.png
Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_6.png


In [6]:
import os
from google import genai

client = genai.Client()

# Folder where images are stored (example: "output_images/upload_nic_2018_5c94b4f64e25f")
folder_path = r"output_images\upload_nic_2018_5c94b4f64e25f"

uploaded_files = []

# Loop through all image files in folder
for filename in sorted(os.listdir(folder_path)):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):  # only images
        file_path = os.path.join(folder_path, filename)
        file = client.files.upload(file=file_path)
        uploaded_files.append(file)
        print(f"✅ Uploaded {filename}, URI: {file.uri}")

print(f"\nTotal uploaded: {len(uploaded_files)} files")


✅ Uploaded page_1.png, URI: https://generativelanguage.googleapis.com/v1beta/files/m14ek7vxsz4y
✅ Uploaded page_2.png, URI: https://generativelanguage.googleapis.com/v1beta/files/ek7klgsf3dgv
✅ Uploaded page_3.png, URI: https://generativelanguage.googleapis.com/v1beta/files/fis5jzol7gwf
✅ Uploaded page_4.png, URI: https://generativelanguage.googleapis.com/v1beta/files/0yueu42a7he1
✅ Uploaded page_5.png, URI: https://generativelanguage.googleapis.com/v1beta/files/5ofwtbd9290y
✅ Uploaded page_6.png, URI: https://generativelanguage.googleapis.com/v1beta/files/m168l62lyamt

Total uploaded: 6 files


In [7]:
print(uploaded_files)

[File(
  create_time=datetime.datetime(2025, 10, 2, 16, 28, 54, 472154, tzinfo=TzInfo(UTC)),
  expiration_time=datetime.datetime(2025, 10, 4, 16, 28, 53, 948965, tzinfo=TzInfo(UTC)),
  mime_type='image/png',
  name='files/m14ek7vxsz4y',
  sha256_hash='NzZmMzVhZWNhMDhmZGJjMTM4OGFhOTY5OTA0OTMyY2Y1NWEwNDQwMTI4NzAwNTFlMzI5MjM5Y2U5MDIwM2RhMA==',
  size_bytes=6952100,
  source=<FileSource.UPLOADED: 'UPLOADED'>,
  state=<FileState.ACTIVE: 'ACTIVE'>,
  update_time=datetime.datetime(2025, 10, 2, 16, 28, 54, 472154, tzinfo=TzInfo(UTC)),
  uri='https://generativelanguage.googleapis.com/v1beta/files/m14ek7vxsz4y'
), File(
  create_time=datetime.datetime(2025, 10, 2, 16, 29, 0, 935388, tzinfo=TzInfo(UTC)),
  expiration_time=datetime.datetime(2025, 10, 4, 16, 29, 0, 301560, tzinfo=TzInfo(UTC)),
  mime_type='image/png',
  name='files/ek7klgsf3dgv',
  sha256_hash='MDMzZTM3N2Y0MWEyODI0Yjc1NTQ1NzFjYTg5NzdjMDQyYzNhMTQ2ZTU4MGVmMzNmYWIyZGJmNzM2NzNlZDQyMg==',
  size_bytes=8487540,
  source=<FileSource.UPLOA

In [3]:
from google import genai

client = genai.Client()

# Directly pass file URIs to the request
file_uris = [
    "https://generativelanguage.googleapis.com/v1beta/files/m14ek7vxsz4y",
    "https://generativelanguage.googleapis.com/v1beta/files/ek7klgsf3dgv",
    "https://generativelanguage.googleapis.com/v1beta/files/fis5jzol7gwf",
    "https://generativelanguage.googleapis.com/v1beta/files/0yueu42a7he1",
    "https://generativelanguage.googleapis.com/v1beta/files/5ofwtbd9290y",
    "https://generativelanguage.googleapis.com/v1beta/files/m168l62lyamt",
]

# Build contents: wrap URIs properly
contents = [{"file_data": {"file_uri": uri}} for uri in file_uris]

# Add your instruction at the end
contents.append(
    """These 6 images are pages of the same inspection document. 
    Extract the following information:
    1. Document type (Gas_safety, EPC, or EIRC)
    2. Address of the inspection
    3. Inspection status (Passed or Failed)

    Return the result strictly as JSON with keys: 
    { "document_type": "", "address": "", "inspection_status": "" }
    """
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
)

print("✅ Document Type:", response.text)


✅ Document Type: ```json
{
  "document_type": "EIRC",
  "address": "42 CHESTERFIELD COURT FLAT 1",
  "inspection_status": "Failed"
}
```


In [15]:
  from google import genai
  from google.genai import types

  with open('page_1.png', 'rb') as f:
      image_bytes = f.read()

  client = genai.Client()
  response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[
      types.Part.from_bytes(
        data=image_bytes,
        mime_type='image/png',
      ),
      'Caption this image.'
    ]
  )

  print(response.text)

This is an "Electrical Installation Condition Report" for a landlord safety inspection, dated August 28, 2018.

**Key details from the report include:**

*   **Report No.:** DPN7/0393125
*   **Contractor's Reference Number:** CRN/007
*   **Client:** David Harouni, located at 13A West Hampstead Mews, NW6 3BB.
*   **Installation Location (Occupier):** David Harouni, 42 Chesterfield Court Flat 1, NW11 8AU.
*   **Type of Installation:** Ticked as "Highway Installation" (though the addresses are residential).
*   **Purpose of the Report:** Landlord Safety Inspection.
*   **Date of Inspection:** 28-8-2018.
*   **Estimated Age of Installation:** 20 years, with evidence of 10-year-old alterations.
*   **Records:** Unknown previous inspection date, no previous certificate/report number, no records available.
*   **Extent of Inspection:** 60% of accessories removed for visual inspection, all final circuits tested.
*   **Operational Limitations:** Insulation resistance between live conductors not

In [14]:
# without structure output
from google import genai
from google.genai import types
import pathlib

client = genai.Client()

# Retrieve and encode the PDF byte
filepath = pathlib.Path(r"C:\Users\Raghul mani M\Downloads\compliance_docs\compliance_docs\upload_nic_2018_5c94b4f64e25f.pdf")

response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      """ 
      Extract the following information: Document type (Gas_safety, EPC, or EIRC),Address of the inspection,Inspection status (Passed or Failed)
    """])
print(response.text)

- **Document type:** EIRC
- **Address of the inspection:** 42 CHESTERFIELD COURT FLAT 1, NNII SAU
- **Inspection status:** Failed


In [11]:
# with structured output
from google import genai
from google.genai import types
import pathlib
from pydantic import BaseModel

class Recipe(BaseModel):
    doc_type: str
    address: str
    inspection:bool

client = genai.Client()

filepath = pathlib.Path(r"C:\Users\Raghul mani M\Downloads\compliance_docs\compliance_docs\upload_nic_2018_5c94b4f64e25f.pdf")

response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      """ 
      Extract the following information:
    1. Document type (Gas_safety, EPC, or EIRC)
    2. Address of the inspection
    3. Inspection status (Passed or Failed)"""],
    config={
        "response_mime_type": "application/json",
        "response_schema": Recipe,
    }
)

print(response)
structured_result: Recipe = response.parsed


sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text='{"doc_type": "EIRC", "address": "42 CHESTERFIELD COURT FLAT 1 NNII SAU", "inspection": false}'
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='u0vjaKm8OK2M1e8PruCxmAk' usage_metadata=GenerateContentResponseUsageMetadata(
  cache_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=44
    ),
    ModalityTokenCount(
      modality=<MediaModality.DOCUMENT: 'DOCUMENT'>,
      token_count=1544
    ),
  ],
  cached_content_token_count=1588,
  candidates_token_count=32,
  prompt_token_count=1593,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=45
    ),
    ModalityTokenCount(
      modality=<MediaModality.DOCUMEN

In [12]:
structured_result.doc_type

'EIRC'

In [13]:
structured_result.inspection

False